In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [ ]:
def load_data(file_path):
    data = pd.read_csv(file_path, header=0, low_memory=False)
    data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S')
    data.set_index('TIMESTAMP', inplace=True, drop=False)
    return data

In [ ]:
def parse_datetime(timestamp):
    try:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        return pd.to_datetime(timestamp, format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
def split_day_night(data):
    day_mask = (data.index.time >= datetime.strptime('08:30', '%H:%M').time()) & \
               (data.index.time <= datetime.strptime('17:30', '%H:%M').time())
    night_mask = (data.index.time >= datetime.strptime('21:30', '%H:%M').time()) | \
                 (data.index.time <= datetime.strptime('05:30', '%H:%M').time())
    day_data = data[day_mask]
    night_data = data[night_mask]
    return day_data, night_data

In [ ]:
def calculate_metrics(data):
    data['10min'] = data.index.floor('10T')
    grouped_data = data.groupby('10min').mean()
    
    aligned_avg_data_u = grouped_data['Ux'].reindex(data.index, method='ffill')
    aligned_avg_data_v = grouped_data['Uy'].reindex(data.index, method='ffill')
    aligned_avg_data_w = grouped_data['Uz'].reindex(data.index, method='ffill')
    
    # Calculate MKE
    MKE = 0.5 * (aligned_avg_data_u**2 + aligned_avg_data_v**2 + aligned_avg_data_w**2)
    
    # Calculate fluctuations
    fluctuation_u = data['Ux'] - aligned_avg_data_u
    fluctuation_v = data['Uy'] - aligned_avg_data_v
    fluctuation_w = data['Uz'] - aligned_avg_data_w
    
    # Calculate TKE
    TKE = 0.5 * (fluctuation_u**2 + fluctuation_v**2 + fluctuation_w**2)
    
    # Calculate turbulence intensity
    turbulence_intensity = TKE / MKE
    
    return MKE, TKE, turbulence_intensity, fluctuation_u, fluctuation_v, fluctuation_w

In [ ]:
def calculate_wind_direction(Ux_avg, Uy_avg):
    angle = np.arctan2(Uy_avg, Ux_avg)
    angle_deg = np.degrees(angle)
    angle_deg = (angle_deg + 360) % 360
    return angle_deg

In [ ]:
def wind_direction_for_time_range(data, start_time, end_time):
    data['10min'] = data.index.floor('10T')
    grouped_data = data.groupby('10min').mean()
    wind_direction = calculate_wind_direction(grouped_data['Ux'], grouped_data['Uy'])
    wind_direction_filtered = wind_direction[(wind_direction.index >= start_time) & (wind_direction.index <= end_time)]
    return wind_direction_filtered


In [ ]:
def plot_wind_direction(data, start_time, end_time):
    wind_direction = wind_direction_for_time_range(data, start_time, end_time)
    fig_wind_dir = px.scatter(x=wind_direction.index, y=wind_direction, title='Wind Direction', labels={'x': 'Time', 'y': 'Wind Direction (degrees)'})
    fig_wind_dir.update_layout(title={'font': {'size': 24}}, xaxis={'title': {'font': {'size': 20}}}, yaxis={'title': {'font': {'size': 20}}}, font={'size': 16})
    fig_wind_dir.show()

In [ ]:
def plot_metrics(data, MKE, TKE, turbulence_intensity, fluctuation_u, fluctuation_v, fluctuation_w, title_suffix, start_timestamp, end_timestamp):
    filtered_data = data[(data['TIMESTAMP'] >= start_timestamp) & (data['TIMESTAMP'] <= end_timestamp)]

    fig_U = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_u[filtered_data.index], title=f'U Fluctuation {title_suffix}')
    fig_U.show()

    fig_V = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_v[filtered_data.index], title=f'V Fluctuation {title_suffix}')
    fig_V.show()

    fig_W = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_w[filtered_data.index], title=f'W Fluctuation {title_suffix}')
    fig_W.show()

    turbulence_intensity_filtered = turbulence_intensity[(turbulence_intensity.index >= start_timestamp) & (turbulence_intensity.index <= end_timestamp)]
    fig_turbulence_intensity = px.line(x=turbulence_intensity_filtered.index, y=turbulence_intensity_filtered, title=f'Turbulence Intensity {title_suffix}')
    fig_turbulence_intensity.show() 

In [ ]:
def plot_turbulence_vs_wind_direction(data, start_date, end_date):
    wind_direction_all = []
    turbulence_intensity_all = []

    current_date = start_date
    while current_date <= end_date:
        next_date = current_date + timedelta(days=1)
        start_timestamp = current_date + timedelta(hours=21, minutes=30)
        end_timestamp = next_date + timedelta(hours=5, minutes=30)
        
        MKE, TKE, turbulence_intensity, _, _, _ = calculate_metrics(data)
        
        wind_direction = wind_direction_for_time_range(data, start_timestamp, end_timestamp)
        
        wind_direction_all.extend(wind_direction)
        turbulence_intensity_all.extend(turbulence_intensity[(turbulence_intensity.index >= start_timestamp) & (turbulence_intensity.index <= end_timestamp)])
        
        current_date += timedelta(days=1)

    fig_turbulence_vs_wind_dir = px.scatter(
        x=wind_direction_all, 
        y=turbulence_intensity_all, 
        title='Turbulence Intensity vs Wind Direction for Multiple Nights', 
        labels={'x': 'Wind Direction (degrees)', 'y': 'Turbulence Intensity (TKE/MKE)'}
    )
    
    fig_turbulence_vs_wind_dir.show()

In [35]:
data = pd.read_csv("flux_tower_15days.csv", header=0, low_memory=False)
data['TIMESTAMP'] = data['TIMESTAMP'].apply(parse_datetime)
data.set_index('TIMESTAMP', inplace=True, drop=False)

# Calculate metrics
MKE, TKE, turbulence_intensity, fluctuation_u, fluctuation_v, fluctuation_w = calculate_metrics(data)

# Plot for a specific night
start = pd.to_datetime('9 June 2015 21:30:00')
end = pd.to_datetime('10 June 2015 5:30:00')
filtered_data = data[(data.index >= start) & (data.index <= end)]
plot_metrics(filtered_data, MKE, TKE, turbulence_intensity, fluctuation_u, fluctuation_v, fluctuation_w, 'Night of 9 June 2015', start, end)

# Plot for multiple nights
start_date = pd.to_datetime('1 June 2015')
end_date = pd.to_datetime('15 June 2015')
plot_turbulence_vs_wind_direction(data, start_date, end_date)